<a href="https://colab.research.google.com/github/sandrakaku/ml0930/blob/master/transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 先解壓縮
import zipfile
f = zipfile.ZipFile("drive/My Drive/train.zip")
f.extractall()

In [ ]:
import pandas as pd
import glob
dogs = glob.glob("train/dog.*")
cats = glob.glob("train/cat.*")
df = pd.DataFrame({  # 準備路徑
    "path":dogs + cats,
    "target":[0] * len(dogs) +[1] * len(cats)
}) 
df

In [ ]:
import random
from tensorflow.keras.preprocessing.image import load_img
idx = random.randint(0, 24999)
load_img(df["path"][idx], target_size=(224, 224))

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
x, y = np.array(df["path"]), np.array(df["target"])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# 學習偷模型
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout # 常用
cnn = vgg16.VGG16(include_top=False, input_shape=(224, 224, 3))
# freez一定要在Compile之前(遷移學習)
for l in cnn.layers: # Dense前都freeze(固定)住
  l.trainable = False

mlp = [
    Flatten(),
    Dense(128, activation="relu"),
    Dense(2, activation="softmax")
]
model = Sequential(cnn.layers + mlp)
model.summary()